# Engine
> Execution spannerlog commands

In [ ]:
#| default_exp engine

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from abc import ABC, abstractmethod
import pytest

from pathlib import Path
from typing import no_type_check, Set, Sequence, Any,Optional,List,Callable,Dict,Union
from pydantic import BaseModel
import networkx as nx
import itertools
from graph_rewrite import draw, draw_match, rewrite, rewrite_iter

## Basic Datatypes

In [ ]:
#| export
from enum import Enum
from typing import Any
from pydantic import ConfigDict

class Span(BaseModel):
    start: int
    end: int

    def __lt__(self, other) -> bool:
        if self.start == other.start:
            return self.end < other.end

        return self.start < other.start

    # # used for sorting `Span`s in dataframes
    # def __hash__(self) -> int:
    #     return hash((self.start, self.end))

class Var(BaseModel):
    name: str
    def __hash__(self):
        return hash(self.name)

class FreeVar(BaseModel):
    name: str
    def __hash__(self):
        return hash(self.name)

PrimitiveType=Union[str,int,Span]
Type = Union[PrimitiveType,Var,FreeVar]

class RelationDefinition(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    scheme: List[type]

class Relation(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    terms: List[Type]

class IEFunction(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    in_schema: List[type]
    out_schema: List[type]
    func: Callable


class IERelation(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    name: str
    in_terms: List[Type]
    out_terms: List[Type]
    def __hash__(self):
        hash_str = f'''{self.name}_in_{'_'.join([str(x) for x in self.in_terms])}_out_{'_'.join([str(x) for x in self.out_terms])}'''
        return hash(hash_str)
class Rule(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    head: Relation
    body: List[Union[Relation,IERelation]]

In [ ]:
#| export
def pretty(obj):
    """pretty printing dataclasses for user messages,
    making them look like spannerlog code instead of python code"""
    
    if isinstance(obj,Span):
        return f"[{obj.start},{obj.end})"
    elif isinstance(obj,(Var,FreeVar)):
        return obj.name
    elif isinstance(obj,RelationDefinition):
        return f"{obj.name}({','.join(pretty(o) for o in obj.scheme)})"
    elif isinstance(obj,Relation):
        return f"{obj.name}({','.join(pretty(o) for o in obj.terms)})"
    elif isinstance(obj,IERelation):
        return f"{obj.name}({','.join(pretty(o) for o in obj.in_terms)}) -> ({','.join(pretty(o) for o in obj.out_terms)})"
    elif isinstance(obj,IEFunction):
        return f"{obj.name}({','.join(pretty(o) for o in obj.in_schema)}) -> ({','.join(pretty(o) for o in obj.out_schema)})"
    elif isinstance(obj,Rule):
        return f"{pretty(obj.head)} <- {','.join(pretty(o) for o in obj.body)}"
    elif isinstance(obj,type):
        return obj.__name__
    else:
        return str(obj)

In [ ]:
rule = Rule(
    head=Relation(name='R', terms=[FreeVar(name='X'), FreeVar(name='Y'), FreeVar(name='Z')]),
    body=[
        Relation(name='S', terms=[FreeVar(name='X'), Span(start=1,end=4)]),
        IERelation(name='T', in_terms=[FreeVar(name='X'), 1], out_terms=[FreeVar(name='Y'), FreeVar(name='Z')])
    ])
assert pretty(rule) == 'R(X,Y,Z) <- S(X,[1,4)),T(X,1) -> (Y,Z)'

In [ ]:
schema = RelationDefinition(name='R', scheme=[int, str, Span])
assert pretty(schema) == 'R(int,str,Span)'
ie_func_schema = IEFunction(name='f', in_schema=[int, str], out_schema=[str, Span],func=lambda x,y: (y,Span(1,2)))
assert pretty(ie_func_schema) == 'f(int,str) -> (str,Span)'

## Engine

In [ ]:
# TODO FROM HERE, derive engine interface from basic commands

# it should have all state needed for execution, the execution itself, and the access to the DB

In [ ]:
# Redesign
# the backend will currently be a dict of dataframes


In [ ]:
class Engine():
    def __init__(self):
        self.symbol_table={
            # key : type,val
        }
        self.Relation_defs={
            # key : RelationDefinition for both real and derived relations
        }
        self.ie_functions={
            # name : IEFunction class
        }

        self.term_graph = nx.Digraph()
        
        node_counter = itertools.Count()

        self.db = {
            # relation_name: dataframe
        }

        # lets skip this for now and keep it a an attribute in the node graph
        # self.rules_to_nodes = {
        #     # rule pretty string, to node id in term_graph
        # }

        # self.rels_to_nodes() = {
        #     # relation name to node that represents it
        # }


    def set_var(var_name,value,read_from_file=False):
        symbol_table = self.symbol_table
        if var_name in symbol_table:
            existing_type,existing_value = symbol_table[var_name]
            if type(value) != existing_type:
                raise ValueError(f"Variable {var_name} was previously defined with {existing_value}({pretty(existing_type)})"
                                f" but is trying to be redefined to {value}({pretty(type(value))}) of a different type which might interfere with previous rule definitions")    
        symbol_table[var_name] = type(value),value
        return 
    def del_var(var_name):
        del self.symbol_table[var_name]

    def set_relation(self,rel_def:RelationDefinition):
        pass

    def del_relation(self,rel_name:str):
        pass

    def add_fact(self,fact:Relation):
        pass

    def del_fact(self,fact:Relation):
        pass

    def set_ie_function(self,ie_func:IEFunction):
        pass

    def del_ie_function(self,name:str):
        pass

    def add_rule(self,rule:Rule):
        pass

    def del_rule(self,name:str):
        pass

    def run_query(self,q:Relation):
        pass
        


In [ ]:
# * Resolve Vars  # lets put this in the term graph
# * Register new Relations
# * Add/remove facts
# * Add rules
# * Run Queries

## DB operations

## SemiNaive Execution